In [ ]:
# CREATE FOOD ANALYSTS

analyst_instructions = """You are creating a team of culinary specialists to research a food/recipe topic.

1. Review the food topic: {topic}

2. Consider feedback: {human_analyst_feedback}

3. Determine important perspectives (recipe, nutrition, dietary modifications, techniques, etc.)

4. Create {max_analysts} culinary specialists, each focusing on different aspects.

Example specialists:
- Recipe Developer (focuses on ingredients and cooking methods)
- Nutritionist (focuses on nutritional value and health benefits)
- Dietary Specialist (focuses on modifications for different diets)
- Cooking Technique Expert (focuses on preparation methods and tips)
"""

def create_food_analysts(state: GenerateAnalystsState):
    topic = state['topic']
    max_analysts = state['max_analysts']
    human_analyst_feedback = state.get('human_analyst_feedback', '')

    structured_llm = llm.with_structured_output(CulinaryPerspectives)
    system_message = analyst_instructions.format(
        topic=topic,
        human_analyst_feedback=human_analyst_feedback,
        max_analysts=max_analysts
    )

    analysts = structured_llm.invoke([
        SystemMessage(content=system_message),
        HumanMessage(content="Generate the culinary specialist analysts.")
    ])

    return {"analysts": analysts.analysts}

def should_continue(state: GenerateAnalystsState):
    if state.get('human_analyst_feedback', None):
        return "create_analysts"
    return END

# Build analyst generation graph
builder = StateGraph(GenerateAnalystsState)
builder.add_node("create_analysts", create_food_analysts)
builder.add_edge(START, "create_analysts")
builder.add_conditional_edges("create_analysts", should_continue, ["create_analysts", END])

memory = MemorySaver()
analyst_graph = builder.compile(checkpointer=memory)

print(" Food analyst generation system ready!")